The goal of this notebook is to develop an end2end text classifier using different embedding like word2vec, fasttext, and doc2vec.

The first pipe is to clean the text data (More cleaning for future data.) 

In [1]:


# Data cleaning
# ===========================================================================
import re
def clean_text(text):
    # replace  . and a space with only a space, then amke all words lower case.
    text = text.replace(". "," ").replace(",","").lower()
    # get rid of the . at the end of each line. 
    cleaned_text = re.sub("\.$","",text)
    
    return cleaned_text
 


class text_clean:
    """
    A class to help with cleaning text data. 
    """
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        assert isinstance(X,pd.Series), "The input data should be pandas Series."
        X = X.apply(clean_text)
        
        return X


# Word embedding training 
# ===========================================================================
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.externals import joblib

from sklearn.base import BaseEstimator

def _find_part_pii(text, model, sep = " "):
    tokenized_text = text.split(sep)
    
    part_pii = model.wv.doesnt_match(tokenized_text)
    
    return part_pii    



def _extracted_pii2matrix(pii_list, model):
    # set the matrix dimensions
    column_num = model.trainables.layer1_size
    row_num = len(pii_list)
    # initialized the matrix
    pii2vec_mat = np.zeros((row_num, column_num))
    # iterate through the pii_list and assign the vectors to matrix.
    for index, ith_pii in enumerate(tqdm(pii_list)):
        pii2vec_mat[index,:] = model.wv[ith_pii]
    
    return pii2vec_mat



class word_embedding(BaseEstimator):
    """
    A class to convert words/docs to vectors by applying any model supported by gensim.  
    
    This class will allow continued training on the pre-trained model by assigning
    the model to the pre_trained option in class initialization.  
    
    After training the model, it will dump the word2vec model to the path assigned to 
    dump_file option.  
    
    
    """
    def __init__(self, algo_name = "word2vec", size = 100, min_count = 1, window = 5, workers =1,\
                 epochs = 5, pre_train = None, dump_file = False, continue_train_pre_train = True,
                 re_train_new_sentences = True):
        
        
        assert algo_name in ["word2vec", 'fasttext', 'doc2vec'], \
        "please enter a model name in ['word2vec', 'fasttext', 'doc2vec']"
        
        self.algo_name = algo_name
        self.epochs = epochs 
        self.pre_train = pre_train
        self.dump_file = dump_file 
        self.re_train_new_sentences = re_train_new_sentences
        self.continue_train_pre_train = continue_train_pre_train
        
        # model options
        self.size = size
        self.min_count = min_count
        self.window = window
        self.workers = workers
        
        
    def _algo_init(self):
        if self.algo_name == "word2vec":
            model = Word2Vec(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "fasttext":
            model = FastText(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "doc2vec":
            model = Doc2Vec(vector_size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
            
        self.model = model
        return self

    def _embedding_training(self, sentences, update = False):
        """
        This helper functions will build the vocabulary, train the model and update the self.model
        with the newly trained model.
        
        if update = True, it will update the vocabulary and the model can continue to train.
        If update = False, the model will rebuild a new vocabulary from scratch using the input data.
        """
        updated_model_with_vocab = self.model

        updated_model_with_vocab.build_vocab(sentences, update = update)
        
        updated_model_with_vocab.train(sentences, total_examples = len(sentences), epochs = self.epochs)
        
        # update the model with the trained one. 
        self.model = updated_model_with_vocab
        
    def _pd_to_gensim_format(self, text):
        
        # special handling for doc2vec model. 
        if self.algo_name == "doc2vec":
            documents = [TaggedDocument(sentence.split(" "), [index])\
                          for index, sentence in enumerate(text)] 
            print("Using index for the tags")    
        else:
            documents = [sentence.split(" ") for sentence in text]
            
            
        return documents
            
        
    def fit(self, X, y = None):
        """
        The fit method will get use the pre_trained model if the model is assigned to the pre_train option.
        
        If the pre_train is None, then the model will be trained. 
        
        
        If the pre_train model is not None, then the default is to continue training on the new model. 
        Unless option continue_train_pre_train is specified as False. The False option will just assign 
        the pre_train model to self.model
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        
        if self.pre_train is not None:
            
            # update the pre_trained model with new training data
            if self.continue_train_pre_train:
                self.model = self.pre_train
                self._embedding_training(sentences = gensim_X, update = True)
                print("continue training with the pre_train model.")
                
            # no training the pre_trained model. 
            elif not self.continue_train_pre_train:
                self.model = self.pre_train
                print("No training with pre_train model.")
                
            
            return self
        
        else:
            # initialize the model, split the sentence into tokens and train it. 
            self._algo_init()
            self._embedding_training(sentences = gensim_X)
            print("Building new vocabulary and training the {} model".format(self.algo_name))
        
        
        # dump the model to disk
        if isinstance(self.dump_file,str):
            self.model.save(self.dump_file)
            print("Writing the {} model to disk.".format(self.algo_name))
            
        return self
        
    
    def transform(self, X):
        """
        If re_train_new_sentences is True, which is the default setting, 
        the model will be re-trained on the new sentences. 
        This will create word embedding for words not in the original vocabulary.
        This will increase the model inference time since it invovles model training. 
        
        For using word2vec to predict PII data, it is recommended to update the model with new sentences. 
        For fastttext, it is not necessary since it will infer from the character n-grams. The fasttext training
        is much longer than word2vec. 
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        
        # update the embedding with new sentences or train the model. 
        if self.re_train_new_sentences:
            self._embedding_training(sentences = gensim_X, update = True)
            print("transforming while training {} model with new data.".format(self.algo_name))
            
            
        # extract the PII 
        extracted_pii_list = [_find_part_pii(text = text, model = self.model)\
                    for text in tqdm(X) ]
        
        # convert the extracted pii text into vectors.
        piivec_matrix = _extracted_pii2matrix(pii_list = extracted_pii_list,\
                                          model = self.model)
        return piivec_matrix 
                                          

In [2]:
import pandas as pd
algo_test_data = pd.read_csv("../data/train_text_with_pii_2019_01_05_02_48_24_796403.csv")

## Word2vec testing

In [3]:
testing_embedding = word_embedding(algo_name = 'word2vec', dump_file = './devs/word2vec.bin')
testing_embedding.fit(algo_test_data['Text']);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix.shape

  0%|          | 0/800 [00:00<?, ?it/s]

Building new vocabulary and training the word2vec model
Writing the word2vec model to disk.
transforming while training word2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 185753.06it/s]


(800, 100)

In [4]:
pre_trained1 = Word2Vec.load("./word2vec/word2vec_cleaned_300_.bin")
pre_trained2 = Word2Vec.load("./devs/word2vec.bin")
testing_pre_trained = word_embedding(algo_name = "word2vec", pre_train = pre_trained2)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

100%|██████████| 800/800 [00:00<00:00, 186185.95it/s]

continue training with the pre_train model.
transforming while training word2vec model with new data.


(800, 100)

## Fasttext testing

In [5]:
testing_embedding = word_embedding(algo_name = 'fasttext', dump_file = "./devs/fasttext.bin")

testing_embedding.fit(algo_test_data["Text"]);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix.shape

Building new vocabulary and training the fasttext model
Writing the fasttext model to disk.


100%|██████████| 800/800 [00:00<00:00, 131529.27it/s]

transforming while training fasttext model with new data.


(800, 100)

In [6]:
pre_trained = FastText.load("./devs/fasttext.bin")
testing_pre_trained = word_embedding(algo_name = "fasttext", pre_train = pre_trained)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

continue training with the pre_train model.


100%|██████████| 800/800 [00:00<00:00, 154857.08it/s]

transforming while training fasttext model with new data.


(800, 100)

## doc2vec testing

In [7]:
testing_embedding_doc2vec = word_embedding(algo_name = 'doc2vec', dump_file = "./devs/doc2vec.bin")
testing_embedding_doc2vec.fit(X = algo_test_data['Text'])

test__doc2vec_pii_matrix = testing_embedding_doc2vec.transform(algo_test_data['Text'])
test__doc2vec_pii_matrix.shape

Using index for the tags


  0%|          | 0/800 [00:00<?, ?it/s]

Building new vocabulary and training the doc2vec model
Writing the doc2vec model to disk.
Using index for the tags
transforming while training doc2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 210451.78it/s]


(800, 100)

In [8]:
pre_trained = FastText.load("./devs/doc2vec.bin")
testing_pre_trained = word_embedding(algo_name = "doc2vec", pre_train = pre_trained)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

Using index for the tags
continue training with the pre_train model.
Using index for the tags


100%|██████████| 800/800 [00:00<00:00, 199538.73it/s]

transforming while training doc2vec model with new data.


(800, 100)

## Add all the pipe above to a single pipeline for training an end-to-end model.

## word2vec pipe

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
logit_clf = LogisticRegression(solver = "lbfgs", max_iter = 10000, class_weight={0:0.9,1:0.1})

In [10]:
word2vec_pipe = Pipeline([('text_cleaning', text_clean()),
                 ("fasttext", word_embedding(algo_name = "word2vec", size= 100,\
                                             dump_file = "./word2vec/word2vec_100_pipe_dump.bin")),
                 ("logit",logit_clf)
                ])

In [11]:
pipe_test_data = pd.read_csv("../clean_data/Cleaned_test_text_with_pii_2018_12_31_05_35_46_815414.csv")

In [12]:
pipe_test_data.head()

,Text,Labels,PII,Cleaned_text,Target
0,Wife marriage cup 7416 Smith Forks before pict...,Address,7416 Smith Forks,wife marriage cup 7416 smith forks before pict...,1
1,However send which. Suite 244 Nice market acce...,Address,Suite 244,however send which suite 244 nice market accep...,1
2,0497 Kemp Lane Amount tough and fire until. Is...,Address,0497 Kemp Lane,0497 kemp lane amount tough and fire until iss...,1
3,Model north receive nature effort 58162 France...,Address,"58162 Frances Shoals Conniemouth, OH 71686",model north receive nature effort 58162 france...,1
4,Child already drive could. Begin such down cel...,Address,538 Gina Circles,child already drive could begin such down cell...,1


In [13]:
%%time 
word2vec_pipe.fit(pipe_test_data["Text"],pipe_test_data['Target'] )

Building new vocabulary and training the word2vec model
Writing the word2vec model to disk.


  1%|          | 730/80000 [00:00<00:10, 7295.33it/s]

transforming while training word2vec model with new data.


100%|██████████| 80000/80000 [00:00<00:00, 273495.78it/s]


CPU times: user 37.3 s, sys: 5.24 s, total: 42.5 s
Wall time: 34.6 s


Pipeline(memory=None,
     steps=[('text_cleaning', <__main__.text_clean object at 0x7f7983150be0>), ('fasttext', word_embedding(algo_name='word2vec', continue_train_pre_train=True,
        dump_file='./word2vec/word2vec_100_pipe_dump.bin', epochs=5,
        min_count=1, pre_train=None, re_train_new_sentences=True, size=100,
...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])

## fasttext pipe

In [14]:
logit_clf = LogisticRegression(solver = "lbfgs", max_iter = 10000, class_weight={0:0.9,1:0.1})
fasttext_pipe = Pipeline([('text_cleaning', text_clean()),
                 ("fasttext", word_embedding(algo_name = "fasttext", size= 100,\
                                             dump_file = "./fasttext/fasttext_100_pipe_dump.bin")),
                 ("logit",logit_clf)
                ])

In [15]:
%%time 
fasttext_pipe.fit(pipe_test_data["Text"],pipe_test_data['Target'] )

Building new vocabulary and training the fasttext model
Writing the fasttext model to disk.


  0%|          | 0/80000 [00:00<?, ?it/s]

transforming while training fasttext model with new data.


100%|██████████| 80000/80000 [00:00<00:00, 237005.16it/s]


CPU times: user 1min 46s, sys: 8.38 s, total: 1min 55s
Wall time: 1min 39s


Pipeline(memory=None,
     steps=[('text_cleaning', <__main__.text_clean object at 0x7f797853d748>), ('fasttext', word_embedding(algo_name='fasttext', continue_train_pre_train=True,
        dump_file='./fasttext/fasttext_100_pipe_dump.bin', epochs=5,
        min_count=1, pre_train=None, re_train_new_sentences=True, size=100,
...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])

## doc2vec pipe

In [16]:
logit_clf = LogisticRegression(solver = "lbfgs", max_iter = 10000, class_weight={0:0.9,1:0.1})
doc2vec_pipe = Pipeline([('text_cleaning', text_clean()),
                 ("doc2vec", word_embedding(algo_name = "doc2vec", size= 100,\
                                             dump_file = "./doc2vec/fasttext_100_pipe_dump.bin")),
                 ("logit",logit_clf)
                ])

In [17]:
%%time 
doc2vec_pipe.fit(pipe_test_data["Text"],pipe_test_data['Target'] )

Using index for the tags
Building new vocabulary and training the doc2vec model
Writing the doc2vec model to disk.
Using index for the tags


  1%|          | 751/80000 [00:00<00:10, 7508.63it/s]

transforming while training doc2vec model with new data.


100%|██████████| 80000/80000 [00:00<00:00, 267942.01it/s]


CPU times: user 50.4 s, sys: 4.07 s, total: 54.5 s
Wall time: 48.1 s


Pipeline(memory=None,
     steps=[('text_cleaning', <__main__.text_clean object at 0x7f79784e3b00>), ('doc2vec', word_embedding(algo_name='doc2vec', continue_train_pre_train=True,
        dump_file='./doc2vec/fasttext_100_pipe_dump.bin', epochs=5,
        min_count=1, pre_train=None, re_train_new_sentences=True, size=100,
   ...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])

# put the above pipe into a CV. 

In [18]:
from scipy.stats import randint, uniform
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV


logit_clf = LogisticRegression(solver = "lbfgs", max_iter = 10000, class_weight={0:0.9,1:0.1})

diff_pipe = Pipeline([('text_cleaning', text_clean()),
                 ("word_embedding", word_embedding()),
                 ("logit",logit_clf)
                ])


param_grid = {
    'word_embedding__algo_name':['word2vec', 'doc2vec','fasttext'],
    'word_embedding__size':[100,200,300],   
    'logit__C': uniform(0,10),
    'logit__class_weight':[{0: 0.9, 1: 0.1}, {0: 0.8, 1: 0.2}, None]
}

diff_pipe_cv = RandomizedSearchCV(estimator = diff_pipe,param_distributions = param_grid, cv =2, \
                                  error_score = 0,n_iter = 10 , scoring = 'f1'
                               ,return_train_score=True, n_jobs = 1)

In [ ]:
%%time 
diff_pipe_cv.fit(pipe_test_data["Text"],pipe_test_data['Target'] )

Building new vocabulary and training the word2vec model


  2%|▏         | 839/40000 [00:00<00:04, 8385.23it/s]

transforming while training word2vec model with new data.


  1%|          | 421/40000 [00:00<00:09, 4205.96it/s]

transforming while training word2vec model with new data.


...

100%|██████████| 40000/40000 [00:00<00:00, 224031.96it/s]


Building new vocabulary and training the fasttext model


  0%|          | 0/40000 [00:00<?, ?it/s]

transforming while training fasttext model with new data.


100%|██████████| 40000/40000 [00:00<00:00, 213128.18it/s]
